# Assembling the TOP dataset

### Import libraries

In [1]:
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt
import glob
import sys
sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config

## Read data into pandas dataframe

How do we define which files should be used together?
options:
- all files in folder
- based on suffix (e.g. "n=895_06-Feb-2023_PVC2.tsv")
- check first columns to see whether it matches

For now, we will will use the first option, approved by Mathijs on 26th June 2023

In [2]:
experiment_folder= 'TOP'

In [3]:
config = Config()
root_directory = config.get_directory('raw_data')
if os.path.isdir(os.path.join(root_directory, experiment_folder)):
    print("this folder exists, we will take tsv from here")
else: 
    print("this folder does not seem to exist, try typing again")

this folder exists, we will take tsv from here


In [4]:
root_directory = config.get_directory('raw_data')
tsv_path = os.path.join(root_directory, experiment_folder)

tsv_files = [os.path.join(tsv_path, file) for file in os.listdir(tsv_path) if file.endswith('.tsv')]
tsv_files

['C:/Projects/brainspin/not_pushed/data_anonymized/TOP\\CoV_qCBF_StandardSpace_DeepWM_n=895_06-Feb-2023_PVC2.tsv',
 'C:/Projects/brainspin/not_pushed/data_anonymized/TOP\\CoV_qCBF_StandardSpace_Tatu_ACA_MCA_PCA_n=895_06-Feb-2023_PVC2.tsv',
 'C:/Projects/brainspin/not_pushed/data_anonymized/TOP\\CoV_qCBF_StandardSpace_Tatu_ACA_MCA_PCA_Prox_Med_Dist_n=895_06-Feb-2023_PVC2.tsv',
 'C:/Projects/brainspin/not_pushed/data_anonymized/TOP\\CoV_qCBF_StandardSpace_TotalGM_n=895_06-Feb-2023_PVC2.tsv',
 'C:/Projects/brainspin/not_pushed/data_anonymized/TOP\\mean_qCBF_StandardSpace_DeepWM_n=895_06-Feb-2023_PVC2.tsv',
 'C:/Projects/brainspin/not_pushed/data_anonymized/TOP\\mean_qCBF_StandardSpace_Tatu_ACA_MCA_PCA_n=895_06-Feb-2023_PVC2.tsv',
 'C:/Projects/brainspin/not_pushed/data_anonymized/TOP\\mean_qCBF_StandardSpace_Tatu_ACA_MCA_PCA_Prox_Med_Dist_n=895_06-Feb-2023_PVC2.tsv',
 'C:/Projects/brainspin/not_pushed/data_anonymized/TOP\\mean_qCBF_StandardSpace_TotalGM_n=895_06-Feb-2023_PVC2.tsv']

In [5]:
# Read files in selected folder into dataframes
dataframes = [pd.read_csv(file, sep='\t', header=[0]) for file in tsv_files]
# make a sample
sample_df = dataframes[2]
cols = sample_df.columns
#look at sample
sample_df

,participant_id,session,LongitudinalTimePoint,SubjectNList,Site,GM_vol,WM_vol,CSF_vol,GM_ICVRatio,GMWM_ICVRatio,...,MCA_distal_R,PCA_proximal_B,PCA_proximal_L,PCA_proximal_R,PCA_intermediate_B,PCA_intermediate_L,PCA_intermediate_R,PCA_distal_B,PCA_distal_L,PCA_distal_R
0,StudyID,...,integer,integer,integer,Liter,Liter,Liter,ratio GM/ICV,ratio (GM+WM)/ICV,...,SD/mean,SD/mean,SD/mean,SD/mean,SD/mean,SD/mean,SD/mean,SD/mean,SD/mean,SD/mean
1,sub-0001_1,ASL_1,1,1,1,0.71736,0.52803,0.31812,0.45881,0.79653,...,3.8698,2.204,1.6071,1.9821,3.3182,2.708,2.5718,3.7916,4.4524,2.4356
2,sub-0002_1,ASL_1,1,2,1,0.72383,0.62394,0.25673,0.45112,0.83999,...,2.0684,1.4706,1.2979,1.6915,1.7094,1.6943,1.6923,1.8967,2.1686,1.4425
3,sub-0005_1,ASL_1,1,3,1,0.71919,0.44499,0.39444,0.46143,0.74693,...,2.6392,1.5148,1.5758,1.4414,1.9507,2.1037,1.7662,2.3456,2.4105,1.9492
4,sub-0006_1,ASL_1,1,4,1,0.64079,0.52942,0.27159,0.44444,0.81163,...,1.7199,1.5141,1.3716,1.599,1.974,2.0258,1.7491,2.0234,1.9595,2.0458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,sub-1165_1,ASL_1,1,891,1,0.71537,0.48601,0.27123,0.48578,0.81582,...,2.0635,1.4744,1.4365,1.4993,1.664,1.7495,1.5787,1.7658,1.6867,1.7406
892,sub-1166_1,ASL_1,1,892,1,0.68938,0.52763,0.26509,0.46514,0.82114,...,2.9425,1.7197,1.7386,1.6567,2.1543,2.2411,1.9333,2.2041,2.0809,2.1718
893,sub-1167_1,ASL_1,1,893,1,0.73473,0.53549,0.35985,0.45074,0.77924,...,2.3496,1.6419,1.7698,1.4954,2.4364,2.577,2.2646,1.9965,1.9663,1.9066
894,sub-1168_1,ASL_1,1,894,1,0.71094,0.60974,0.3748,0.41931,0.77894,...,2.7611,1.6238,1.3945,1.9383,2.4139,2.2668,2.4277,2.7811,2.8263,2.5794


## Below we will see how many different columns should be in our final dataframe except sex and age, which we add from anotehr frame

In [6]:
list_all_column = []
for frame in dataframes:
    list_columns = frame.columns.to_list()
    for x in list_columns:
        if x not in list_all_column:
            list_all_column.append(x)
len(list_all_column)

54

In [7]:
#len(list_all_column)

### Below we take all the dataframes and put them into tsv files in a specific folder

In [8]:
numbr = 0
for frame in dataframes:
    filepath = '../open_work/internal_results/stitchy' 
    filename = os.path.join(filepath,str(numbr+1)) 
    if not os.path.exists(filepath):
    # if filder doesn't exist, create it
        os.makedirs(filepath)
    frame.to_csv((filename +'.tsv'), sep="\t")
    numbr +=1

## Copy identical columns from any file

we could also read it from the data, but if it's always the same, we can just define it here

In [9]:
tsv_folder_made = '../open_work/internal_results/stitchy'
identical_columns = sep.check_identical_columns(tsv_folder_made)

In [10]:
identical_columns

['session',
 'GMWM_ICVRatio',
 'GM_ICVRatio',
 'GM_vol',
 'CSF_vol',
 'Site',
 'WMH_vol',
 'LongitudinalTimePoint',
 'SubjectNList',
 'participant_id',
 'WM_vol',
 'WMH_count']

In [11]:
#sample_df.columns

In [12]:
stitched = sample_df[identical_columns].copy()

In [13]:
n_identical = stitched.shape[1]

In [14]:
stitched

,session,GMWM_ICVRatio,GM_ICVRatio,GM_vol,CSF_vol,Site,WMH_vol,LongitudinalTimePoint,SubjectNList,participant_id,WM_vol,WMH_count
0,...,ratio (GM+WM)/ICV,ratio GM/ICV,Liter,Liter,integer,mL,integer,integer,StudyID,Liter,n lesions (integer)
1,ASL_1,0.79653,0.45881,0.71736,0.31812,1,1.743,1,1,sub-0001_1,0.52803,24
2,ASL_1,0.83999,0.45112,0.72383,0.25673,1,1.629,1,2,sub-0002_1,0.62394,23
3,ASL_1,0.74693,0.46143,0.71919,0.39444,1,NaN,1,3,sub-0005_1,0.44499,NaN
4,ASL_1,0.81163,0.44444,0.64079,0.27159,1,1.631,1,4,sub-0006_1,0.52942,20
...,...,...,...,...,...,...,...,...,...,...,...,...
891,ASL_1,0.81582,0.48578,0.71537,0.27123,1,1.425,1,891,sub-1165_1,0.48601,18
892,ASL_1,0.82114,0.46514,0.68938,0.26509,1,0.723,1,892,sub-1166_1,0.52763,18
893,ASL_1,0.77924,0.45074,0.73473,0.35985,1,3.935,1,893,sub-1167_1,0.53549,20
894,ASL_1,0.77894,0.41931,0.71094,0.3748,1,4.17,1,894,sub-1168_1,0.60974,26


### Here is where we add the different parts to stitched

In [15]:
for df in dataframes:
    for col in df.columns[n_identical:]:
        stitched[col] = df[col]

stitched.columns

Index(['session', 'GMWM_ICVRatio', 'GM_ICVRatio', 'GM_vol', 'CSF_vol', 'Site',
       'WMH_vol', 'LongitudinalTimePoint', 'SubjectNList', 'participant_id',
       'WM_vol', 'WMH_count', 'DeepWM_B', 'DeepWM_L', 'DeepWM_R', 'ACA_B',
       'ACA_L', 'ACA_R', 'MCA_B', 'MCA_L', 'MCA_R', 'PCA_B', 'PCA_L', 'PCA_R',
       'ACA_proximal_B', 'ACA_proximal_L', 'ACA_proximal_R',
       'ACA_intermediate_B', 'ACA_intermediate_L', 'ACA_intermediate_R',
       'ACA_distal_B', 'ACA_distal_L', 'ACA_distal_R', 'MCA_proximal_B',
       'MCA_proximal_L', 'MCA_proximal_R', 'MCA_intermediate_B',
       'MCA_intermediate_L', 'MCA_intermediate_R', 'MCA_distal_B',
       'MCA_distal_L', 'MCA_distal_R', 'PCA_proximal_B', 'PCA_proximal_L',
       'PCA_proximal_R', 'PCA_intermediate_B', 'PCA_intermediate_L',
       'PCA_intermediate_R', 'PCA_distal_B', 'PCA_distal_L', 'PCA_distal_R',
       'TotalGM_B', 'TotalGM_L', 'TotalGM_R'],
      dtype='object')

In [16]:
len(stitched.columns)

54

## Here we can should get rid of second visits, but what we see is that session was not in the common columns. We will get rid of all second time points, and people ending in _2. And mention to scientists

In [17]:
#stitched.columns

In [18]:
stitched = stitched[1:]
#stitched

In [19]:
stitched['session'].unique()

array(['ASL_1'], dtype=object)

In [20]:
stitched['LongitudinalTimePoint'].unique()

array(['1'], dtype=object)

### so due the fact they are all on first visit, first session, we can write this into p[atient id LATER

## So in this dataset we have one longitudinal timepoint, and one type of session. no need to filter down away from this

In [21]:
stitched.columns

Index(['session', 'GMWM_ICVRatio', 'GM_ICVRatio', 'GM_vol', 'CSF_vol', 'Site',
       'WMH_vol', 'LongitudinalTimePoint', 'SubjectNList', 'participant_id',
       'WM_vol', 'WMH_count', 'DeepWM_B', 'DeepWM_L', 'DeepWM_R', 'ACA_B',
       'ACA_L', 'ACA_R', 'MCA_B', 'MCA_L', 'MCA_R', 'PCA_B', 'PCA_L', 'PCA_R',
       'ACA_proximal_B', 'ACA_proximal_L', 'ACA_proximal_R',
       'ACA_intermediate_B', 'ACA_intermediate_L', 'ACA_intermediate_R',
       'ACA_distal_B', 'ACA_distal_L', 'ACA_distal_R', 'MCA_proximal_B',
       'MCA_proximal_L', 'MCA_proximal_R', 'MCA_intermediate_B',
       'MCA_intermediate_L', 'MCA_intermediate_R', 'MCA_distal_B',
       'MCA_distal_L', 'MCA_distal_R', 'PCA_proximal_B', 'PCA_proximal_L',
       'PCA_proximal_R', 'PCA_intermediate_B', 'PCA_intermediate_L',
       'PCA_intermediate_R', 'PCA_distal_B', 'PCA_distal_L', 'PCA_distal_R',
       'TotalGM_B', 'TotalGM_L', 'TotalGM_R'],
      dtype='object')

In [22]:
sexage_path = os.path.join(root_directory, 'age_data', 'Age_sex_TOP.csv')
sexage_df = pd.read_csv(sexage_path, index_col=0)
sexage_df['renumber'] = sexage_df.index
sexage_df['renumber'] = sexage_df['renumber'].apply(str)
sexage_df

,TP,Sex,Age,renumber
ID,,,,
sub-0001_1,1,1,43.49,sub-0001_1
sub-0002_1,1,0,38.30,sub-0002_1
sub-0019_1,1,1,32.30,sub-0019_1
sub-0020_1,1,0,21.97,sub-0020_1
sub-0022_1,1,0,37.52,sub-0022_1
...,...,...,...,...
sub-1163_1,1,0,19.06,sub-1163_1
sub-1165_1,1,0,33.86,sub-1165_1
sub-1167_1,1,1,33.92,sub-1167_1


## Here we take the patient ID and align it with our other frame's index

In [23]:
stitched = stitched.reset_index(drop=False)
stitched

,index,session,GMWM_ICVRatio,GM_ICVRatio,GM_vol,CSF_vol,Site,WMH_vol,LongitudinalTimePoint,SubjectNList,...,PCA_proximal_R,PCA_intermediate_B,PCA_intermediate_L,PCA_intermediate_R,PCA_distal_B,PCA_distal_L,PCA_distal_R,TotalGM_B,TotalGM_L,TotalGM_R
0,1,ASL_1,0.79653,0.45881,0.71736,0.31812,1,1.743,1,1,...,57.7411,65.9906,56.1429,76.2191,83.605,71.446,96.2028,60.6981,57.922,63.8339
1,2,ASL_1,0.83999,0.45112,0.72383,0.25673,1,1.629,1,2,...,60.472,71.5105,73.7114,69.7295,90.1896,93.376,85.1374,71.6047,74.1927,68.7649
2,3,ASL_1,0.74693,0.46143,0.71919,0.39444,1,NaN,1,3,...,72.2354,84.4929,86.6735,82.4922,100.5462,105.0117,94.6801,82.4797,82.9234,81.9457
3,4,ASL_1,0.81163,0.44444,0.64079,0.27159,1,1.631,1,4,...,55.3051,61.0346,63.839,59.1239,66.4437,68.8396,64.2212,63.7201,65.3173,62.0529
4,5,ASL_1,0.79678,0.45686,0.70341,0.31289,1,0.681,1,5,...,78.5647,92.5715,91.9522,93.0762,113.2723,114.2614,112.1711,93.0743,94.7247,91.2521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,891,ASL_1,0.81582,0.48578,0.71537,0.27123,1,1.425,1,891,...,65.4647,74.8321,77.6994,72.8863,85.63,92.9272,79.7285,77.103,76.9157,77.3044
891,892,ASL_1,0.82114,0.46514,0.68938,0.26509,1,0.723,1,892,...,42.1407,49.1094,48.9355,49.2011,58.2895,59.8466,56.0928,54.525,55.3724,53.5625
892,893,ASL_1,0.77924,0.45074,0.73473,0.35985,1,3.935,1,893,...,42.9568,49.334,49.8061,48.8532,67.8209,68.9911,65.9914,54.4204,54.3193,54.5318
893,894,ASL_1,0.77894,0.41931,0.71094,0.3748,1,4.17,1,894,...,38.3813,46.5101,52.5951,41.1785,49.5986,51.929,47.0285,51.4417,52.1897,50.5664


In [24]:
len(stitched.participant_id.unique())

895

In [25]:
stitched['renumber'] = stitched['participant_id']

In [26]:
sexage_df = sexage_df.reset_index(drop=True)
sexage_df

,TP,Sex,Age,renumber
0,1,1,43.49,sub-0001_1
1,1,0,38.30,sub-0002_1
2,1,1,32.30,sub-0019_1
3,1,0,21.97,sub-0020_1
4,1,0,37.52,sub-0022_1
...,...,...,...,...
526,1,0,19.06,sub-1163_1
527,1,0,33.86,sub-1165_1
528,1,1,33.92,sub-1167_1
529,1,1,45.31,sub-1168_1


In [27]:
#stitched.columns

In [28]:
result = stitched.merge(sexage_df, on="renumber")
result

,index,session,GMWM_ICVRatio,GM_ICVRatio,GM_vol,CSF_vol,Site,WMH_vol,LongitudinalTimePoint,SubjectNList,...,PCA_distal_B,PCA_distal_L,PCA_distal_R,TotalGM_B,TotalGM_L,TotalGM_R,renumber,TP,Sex,Age
0,1,ASL_1,0.79653,0.45881,0.71736,0.31812,1,1.743,1,1,...,83.605,71.446,96.2028,60.6981,57.922,63.8339,sub-0001_1,1,1,43.49
1,2,ASL_1,0.83999,0.45112,0.72383,0.25673,1,1.629,1,2,...,90.1896,93.376,85.1374,71.6047,74.1927,68.7649,sub-0002_1,1,0,38.30
2,15,ASL_1,0.78753,0.45046,0.71224,0.33594,1,0.621,1,15,...,93.0546,87.1016,101.595,74.0588,73.2642,74.9254,sub-0019_1,1,1,32.30
3,16,ASL_1,0.80968,0.49124,0.67042,0.25974,1,0.736,1,16,...,96.3119,94.6079,98.0099,79.1128,79.1545,79.071,sub-0020_1,1,0,21.97
4,18,ASL_1,0.83319,0.4796,0.63666,0.22144,1,2.009,1,18,...,88.7337,91.4656,86.0431,73.3492,73.4193,73.2958,sub-0022_1,1,0,37.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,889,ASL_1,0.83842,0.51347,0.78201,0.24608,1,1.431,1,889,...,115.6876,128.9588,102.7405,96.4466,99.4881,93.2525,sub-1163_1,1,0,19.06
527,891,ASL_1,0.81582,0.48578,0.71537,0.27123,1,1.425,1,891,...,85.63,92.9272,79.7285,77.103,76.9157,77.3044,sub-1165_1,1,0,33.86
528,893,ASL_1,0.77924,0.45074,0.73473,0.35985,1,3.935,1,893,...,67.8209,68.9911,65.9914,54.4204,54.3193,54.5318,sub-1167_1,1,1,33.92
529,894,ASL_1,0.77894,0.41931,0.71094,0.3748,1,4.17,1,894,...,49.5986,51.929,47.0285,51.4417,52.1897,50.5664,sub-1168_1,1,1,45.31


In [29]:
result

,index,session,GMWM_ICVRatio,GM_ICVRatio,GM_vol,CSF_vol,Site,WMH_vol,LongitudinalTimePoint,SubjectNList,...,PCA_distal_B,PCA_distal_L,PCA_distal_R,TotalGM_B,TotalGM_L,TotalGM_R,renumber,TP,Sex,Age
0,1,ASL_1,0.79653,0.45881,0.71736,0.31812,1,1.743,1,1,...,83.605,71.446,96.2028,60.6981,57.922,63.8339,sub-0001_1,1,1,43.49
1,2,ASL_1,0.83999,0.45112,0.72383,0.25673,1,1.629,1,2,...,90.1896,93.376,85.1374,71.6047,74.1927,68.7649,sub-0002_1,1,0,38.30
2,15,ASL_1,0.78753,0.45046,0.71224,0.33594,1,0.621,1,15,...,93.0546,87.1016,101.595,74.0588,73.2642,74.9254,sub-0019_1,1,1,32.30
3,16,ASL_1,0.80968,0.49124,0.67042,0.25974,1,0.736,1,16,...,96.3119,94.6079,98.0099,79.1128,79.1545,79.071,sub-0020_1,1,0,21.97
4,18,ASL_1,0.83319,0.4796,0.63666,0.22144,1,2.009,1,18,...,88.7337,91.4656,86.0431,73.3492,73.4193,73.2958,sub-0022_1,1,0,37.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,889,ASL_1,0.83842,0.51347,0.78201,0.24608,1,1.431,1,889,...,115.6876,128.9588,102.7405,96.4466,99.4881,93.2525,sub-1163_1,1,0,19.06
527,891,ASL_1,0.81582,0.48578,0.71537,0.27123,1,1.425,1,891,...,85.63,92.9272,79.7285,77.103,76.9157,77.3044,sub-1165_1,1,0,33.86
528,893,ASL_1,0.77924,0.45074,0.73473,0.35985,1,3.935,1,893,...,67.8209,68.9911,65.9914,54.4204,54.3193,54.5318,sub-1167_1,1,1,33.92
529,894,ASL_1,0.77894,0.41931,0.71094,0.3748,1,4.17,1,894,...,49.5986,51.929,47.0285,51.4417,52.1897,50.5664,sub-1168_1,1,1,45.31


In [30]:
# just in case there are duplicates in there
result = result.loc[:,~result.columns.duplicated()].copy()

In [31]:
result

,index,session,GMWM_ICVRatio,GM_ICVRatio,GM_vol,CSF_vol,Site,WMH_vol,LongitudinalTimePoint,SubjectNList,...,PCA_distal_B,PCA_distal_L,PCA_distal_R,TotalGM_B,TotalGM_L,TotalGM_R,renumber,TP,Sex,Age
0,1,ASL_1,0.79653,0.45881,0.71736,0.31812,1,1.743,1,1,...,83.605,71.446,96.2028,60.6981,57.922,63.8339,sub-0001_1,1,1,43.49
1,2,ASL_1,0.83999,0.45112,0.72383,0.25673,1,1.629,1,2,...,90.1896,93.376,85.1374,71.6047,74.1927,68.7649,sub-0002_1,1,0,38.30
2,15,ASL_1,0.78753,0.45046,0.71224,0.33594,1,0.621,1,15,...,93.0546,87.1016,101.595,74.0588,73.2642,74.9254,sub-0019_1,1,1,32.30
3,16,ASL_1,0.80968,0.49124,0.67042,0.25974,1,0.736,1,16,...,96.3119,94.6079,98.0099,79.1128,79.1545,79.071,sub-0020_1,1,0,21.97
4,18,ASL_1,0.83319,0.4796,0.63666,0.22144,1,2.009,1,18,...,88.7337,91.4656,86.0431,73.3492,73.4193,73.2958,sub-0022_1,1,0,37.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,889,ASL_1,0.83842,0.51347,0.78201,0.24608,1,1.431,1,889,...,115.6876,128.9588,102.7405,96.4466,99.4881,93.2525,sub-1163_1,1,0,19.06
527,891,ASL_1,0.81582,0.48578,0.71537,0.27123,1,1.425,1,891,...,85.63,92.9272,79.7285,77.103,76.9157,77.3044,sub-1165_1,1,0,33.86
528,893,ASL_1,0.77924,0.45074,0.73473,0.35985,1,3.935,1,893,...,67.8209,68.9911,65.9914,54.4204,54.3193,54.5318,sub-1167_1,1,1,33.92
529,894,ASL_1,0.77894,0.41931,0.71094,0.3748,1,4.17,1,894,...,49.5986,51.929,47.0285,51.4417,52.1897,50.5664,sub-1168_1,1,1,45.31


## Conform file to new standard

### example of new standard (from M.D. on 23/08/2023)

In [32]:
standard_path = '../researcher_interface/sample_sep_values/showable_standard.csv'


In [33]:
standard = pd.read_csv(standard_path)
set_standard = set(standard.columns.to_list())

In [34]:
result.columns = result.columns.str.lower()
set_results= set(result.columns.to_list())

z = set_results.intersection(set_standard) 
#z

In [35]:
# # show values that are not as in template, may need recoding
# set_results - z

In [36]:
#standard.columns

In [37]:
#result.columns

In [38]:
# shift column 'Name' to first position
first_column = result.pop('participant_id')
  
# insert column using insert(position,column_name,
# first_column) function
result.insert(0, 'participant_id', first_column)
result['participant_id'] = result['participant_id']+'_ses-1_run-1'
#result

In [39]:
#standard.columns

In [40]:
result['session_id'] = result['session']
second_column = result.pop('session_id')
result.insert(1, 'session_id', second_column)
result['site'] = "TOP"
result['run_id'] = result['longitudinaltimepoint']
#result

In [41]:
#standard.columns

In [42]:
result['gm_icv_ratio'] = result['gm_icvratio']
result['gmwm_icv_ratio'] = result['gm_icvratio']
result['cbf_gm_pvc0'] = result['totalgm_b'] 
result['cbf_gm_pvc2']= 'NaN'
result['cbf_wm_pvc0']= result['deepwm_b']
result['cbf_wm_pvc2']= 'NaN'
result['cbf_aca_pvc0'] =  result['aca_b']
result['cbf_mca_pvc0']  = result['mca_b']
result['cbf_pca_pvc0'] =  result['pca_b']
result['cbf_aca_pvc2']  = 'NaN'
result['cbf_mca_pvc2']  = 'NaN'
result['cbf_pca_pvc2']  = 'NaN'
result['cov_gm_pvc0'] =  'NaN'
result['cov_gm_pvc2']  = 'NaN'
result['cov_wm_pvc0']  = 'NaN'
result['cov_wm_pvc2']  = 'NaN'
result['cov_aca_pvc0'] =  'NaN'
result['cov_mca_pvc0']  = 'NaN'
result['cov_pca_pvc0']  = 'NaN'
result['cov_aca_pvc2'] = 'NaN'
result['cov_mca_pvc2'] = 'NaN'
result['cov_pca_pvc2'] = 'NaN'


     

In [43]:
result.columns


Index(['participant_id', 'session_id', 'index', 'session', 'gmwm_icvratio',
       'gm_icvratio', 'gm_vol', 'csf_vol', 'site', 'wmh_vol',
       'longitudinaltimepoint', 'subjectnlist', 'wm_vol', 'wmh_count',
       'deepwm_b', 'deepwm_l', 'deepwm_r', 'aca_b', 'aca_l', 'aca_r', 'mca_b',
       'mca_l', 'mca_r', 'pca_b', 'pca_l', 'pca_r', 'aca_proximal_b',
       'aca_proximal_l', 'aca_proximal_r', 'aca_intermediate_b',
       'aca_intermediate_l', 'aca_intermediate_r', 'aca_distal_b',
       'aca_distal_l', 'aca_distal_r', 'mca_proximal_b', 'mca_proximal_l',
       'mca_proximal_r', 'mca_intermediate_b', 'mca_intermediate_l',
       'mca_intermediate_r', 'mca_distal_b', 'mca_distal_l', 'mca_distal_r',
       'pca_proximal_b', 'pca_proximal_l', 'pca_proximal_r',
       'pca_intermediate_b', 'pca_intermediate_l', 'pca_intermediate_r',
       'pca_distal_b', 'pca_distal_l', 'pca_distal_r', 'totalgm_b',
       'totalgm_l', 'totalgm_r', 'renumber', 'tp', 'sex', 'age', 'run_id',
       'gm_i

In [44]:
# we added a bunch of columns 
len(result.columns)

83

In [45]:
# set_results= set(result.columns.to_list())

# z = set_results.intersection(set_standard) 
# new_result_columns = []
# for listable in z:
#     new_result_columns.append(listable)
# new_results = result[new_result_columns]
# new_results.columns

In [46]:
# standard.columns[:33]

In [47]:
# new_results = new_results[standard.columns[:33]]
# new_results

In [48]:
sex_mapping = {0:'F',1:'M',}
results = result.assign(sex = result.sex.map(sex_mapping))
results

,participant_id,session_id,index,session,gmwm_icvratio,gm_icvratio,gm_vol,csf_vol,site,wmh_vol,...,cov_gm_pvc0,cov_gm_pvc2,cov_wm_pvc0,cov_wm_pvc2,cov_aca_pvc0,cov_mca_pvc0,cov_pca_pvc0,cov_aca_pvc2,cov_mca_pvc2,cov_pca_pvc2
0,sub-0001_1_ses-1_run-1,ASL_1,1,ASL_1,0.79653,0.45881,0.71736,0.31812,TOP,1.743,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-0002_1_ses-1_run-1,ASL_1,2,ASL_1,0.83999,0.45112,0.72383,0.25673,TOP,1.629,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub-0019_1_ses-1_run-1,ASL_1,15,ASL_1,0.78753,0.45046,0.71224,0.33594,TOP,0.621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub-0020_1_ses-1_run-1,ASL_1,16,ASL_1,0.80968,0.49124,0.67042,0.25974,TOP,0.736,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub-0022_1_ses-1_run-1,ASL_1,18,ASL_1,0.83319,0.4796,0.63666,0.22144,TOP,2.009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,sub-1163_1_ses-1_run-1,ASL_1,889,ASL_1,0.83842,0.51347,0.78201,0.24608,TOP,1.431,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527,sub-1165_1_ses-1_run-1,ASL_1,891,ASL_1,0.81582,0.48578,0.71537,0.27123,TOP,1.425,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528,sub-1167_1_ses-1_run-1,ASL_1,893,ASL_1,0.77924,0.45074,0.73473,0.35985,TOP,3.935,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529,sub-1168_1_ses-1_run-1,ASL_1,894,ASL_1,0.77894,0.41931,0.71094,0.3748,TOP,4.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## here need to reorder the columns again

In [49]:
standard.columns

Index(['participant_id', 'session_id', 'run_id', 'age', 'sex', 'site',
       'gm_vol', 'wm_vol', 'csf_vol', 'gm_icv_ratio', 'gmwm_icv_ratio',
       'wmh_vol', 'wmh_count', 'cbf_gm_pvc0', 'cbf_gm_pvc2', 'cbf_wm_pvc0',
       'cbf_wm_pvc2', 'cbf_aca_pvc0', 'cbf_mca_pvc0', 'cbf_pca_pvc0',
       'cbf_aca_pvc2', 'cbf_mca_pvc2', 'cbf_pca_pvc2', 'cov_gm_pvc0',
       'cov_gm_pvc2', 'cov_wm_pvc0', 'cov_wm_pvc2', 'cov_aca_pvc0',
       'cov_mca_pvc0', 'cov_pca_pvc0', 'cov_aca_pvc2', 'cov_mca_pvc2',
       'cov_pca_pvc2', 'Additional_1', 'Additional_2'],
      dtype='object')

In [50]:
results.columns

Index(['participant_id', 'session_id', 'index', 'session', 'gmwm_icvratio',
       'gm_icvratio', 'gm_vol', 'csf_vol', 'site', 'wmh_vol',
       'longitudinaltimepoint', 'subjectnlist', 'wm_vol', 'wmh_count',
       'deepwm_b', 'deepwm_l', 'deepwm_r', 'aca_b', 'aca_l', 'aca_r', 'mca_b',
       'mca_l', 'mca_r', 'pca_b', 'pca_l', 'pca_r', 'aca_proximal_b',
       'aca_proximal_l', 'aca_proximal_r', 'aca_intermediate_b',
       'aca_intermediate_l', 'aca_intermediate_r', 'aca_distal_b',
       'aca_distal_l', 'aca_distal_r', 'mca_proximal_b', 'mca_proximal_l',
       'mca_proximal_r', 'mca_intermediate_b', 'mca_intermediate_l',
       'mca_intermediate_r', 'mca_distal_b', 'mca_distal_l', 'mca_distal_r',
       'pca_proximal_b', 'pca_proximal_l', 'pca_proximal_r',
       'pca_intermediate_b', 'pca_intermediate_l', 'pca_intermediate_r',
       'pca_distal_b', 'pca_distal_l', 'pca_distal_r', 'totalgm_b',
       'totalgm_l', 'totalgm_r', 'renumber', 'tp', 'sex', 'age', 'run_id',
       'gm_i

In [51]:
len(results.columns)

83

In [52]:
# shift columns to  position
third_column = results.pop('run_id')
fourth_column = results.pop('age')
fifth_column = results.pop('sex')
sixth_column = results.pop('site')
seventh_column = results.pop('gm_vol')
eight_column = results.pop('wm_vol')
ninth_column = results.pop('csf_vol')
tenth_column = results.pop('gm_icv_ratio')
eleventh_column = results.pop('gmwm_icv_ratio')
twelvth_column = results.pop('wmh_vol')
thirteenth_column = results.pop('wmh_count')

#last_column = results.pop('index')

results.insert(2, 'run_id', third_column)
results.insert(3, 'age', fourth_column)
results.insert(4, 'sex', fifth_column)
results.insert(5, 'site', sixth_column)
results.insert(6, 'gm_vol', seventh_column)
results.insert(7, 'wm_vol', eight_column)
results.insert(8, 'csf_vol', ninth_column)
results.insert(9, 'gm_ivc_ratio',tenth_column)
results.insert(10, 'gmwm_ivc_ratio',eleventh_column)
results.insert(11, 'wmh_vol',twelvth_column)
results.insert(12, 'wmh_count',thirteenth_column)
#results.insert(82, 'index', last_column)

results

,participant_id,session_id,run_id,age,sex,site,gm_vol,wm_vol,csf_vol,gm_ivc_ratio,...,cov_gm_pvc0,cov_gm_pvc2,cov_wm_pvc0,cov_wm_pvc2,cov_aca_pvc0,cov_mca_pvc0,cov_pca_pvc0,cov_aca_pvc2,cov_mca_pvc2,cov_pca_pvc2
0,sub-0001_1_ses-1_run-1,ASL_1,1,43.49,M,TOP,0.71736,0.52803,0.31812,0.45881,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-0002_1_ses-1_run-1,ASL_1,1,38.30,F,TOP,0.72383,0.62394,0.25673,0.45112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub-0019_1_ses-1_run-1,ASL_1,1,32.30,M,TOP,0.71224,0.53295,0.33594,0.45046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub-0020_1_ses-1_run-1,ASL_1,1,21.97,F,TOP,0.67042,0.43458,0.25974,0.49124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub-0022_1_ses-1_run-1,ASL_1,1,37.52,F,TOP,0.63666,0.46938,0.22144,0.4796,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,sub-1163_1_ses-1_run-1,ASL_1,1,19.06,F,TOP,0.78201,0.49489,0.24608,0.51347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527,sub-1165_1_ses-1_run-1,ASL_1,1,33.86,F,TOP,0.71537,0.48601,0.27123,0.48578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528,sub-1167_1_ses-1_run-1,ASL_1,1,33.92,M,TOP,0.73473,0.53549,0.35985,0.45074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529,sub-1168_1_ses-1_run-1,ASL_1,1,45.31,M,TOP,0.71094,0.60974,0.3748,0.41931,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
results = results.drop(['index', 'subjectnlist','session','longitudinaltimepoint', 'tp','gmwm_icvratio','gm_icvratio'], axis=1) 

In [54]:
for column in results.columns:
    print(column, type(results[column][9]))

participant_id <class 'str'>
session_id <class 'str'>
run_id <class 'str'>
age <class 'numpy.float64'>
sex <class 'str'>
site <class 'str'>
gm_vol <class 'str'>
wm_vol <class 'str'>
csf_vol <class 'str'>
gm_ivc_ratio <class 'str'>
gmwm_ivc_ratio <class 'str'>
wmh_vol <class 'str'>
wmh_count <class 'str'>
deepwm_b <class 'str'>
deepwm_l <class 'str'>
deepwm_r <class 'str'>
aca_b <class 'str'>
aca_l <class 'str'>
aca_r <class 'str'>
mca_b <class 'str'>
mca_l <class 'str'>
mca_r <class 'str'>
pca_b <class 'str'>
pca_l <class 'str'>
pca_r <class 'str'>
aca_proximal_b <class 'str'>
aca_proximal_l <class 'str'>
aca_proximal_r <class 'str'>
aca_intermediate_b <class 'str'>
aca_intermediate_l <class 'str'>
aca_intermediate_r <class 'str'>
aca_distal_b <class 'str'>
aca_distal_l <class 'str'>
aca_distal_r <class 'str'>
mca_proximal_b <class 'str'>
mca_proximal_l <class 'str'>
mca_proximal_r <class 'str'>
mca_intermediate_b <class 'str'>
mca_intermediate_l <class 'str'>
mca_intermediate_r <class

## now take last 20 columns, and put them after wmh count
not done yet

In [66]:
pd.__version__

'2.0.0'

In [68]:
results['cbf_gm_pvc0'] = results['cbf_gm_pvc0'].shift(20, axis=0)

#for column in results.columns:
    #print(column)
#df[['Age', 'Salary']] = df[['Age', 'Salary']].shift(1)

In [69]:
for column in results.columns:
    print(column)

participant_id
session_id
run_id
age
sex
site
gm_vol
wm_vol
csf_vol
gm_ivc_ratio
gmwm_ivc_ratio
wmh_vol
wmh_count
deepwm_b
deepwm_l
deepwm_r
aca_b
aca_l
aca_r
mca_b
mca_l
mca_r
pca_b
pca_l
pca_r
aca_proximal_b
aca_proximal_l
aca_proximal_r
aca_intermediate_b
aca_intermediate_l
aca_intermediate_r
aca_distal_b
aca_distal_l
aca_distal_r
mca_proximal_b
mca_proximal_l
mca_proximal_r
mca_intermediate_b
mca_intermediate_l
mca_intermediate_r
mca_distal_b
mca_distal_l
mca_distal_r
pca_proximal_b
pca_proximal_l
pca_proximal_r
pca_intermediate_b
pca_intermediate_l
pca_intermediate_r
pca_distal_b
pca_distal_l
pca_distal_r
totalgm_b
totalgm_l
totalgm_r
renumber
cbf_gm_pvc0
cbf_gm_pvc2
cbf_wm_pvc0
cbf_wm_pvc2
cbf_aca_pvc0
cbf_mca_pvc0
cbf_pca_pvc0
cbf_aca_pvc2
cbf_mca_pvc2
cbf_pca_pvc2
cov_gm_pvc0
cov_gm_pvc2
cov_wm_pvc0
cov_wm_pvc2
cov_aca_pvc0
cov_mca_pvc0
cov_pca_pvc0
cov_aca_pvc2
cov_mca_pvc2
cov_pca_pvc2


In [56]:
for column in results.columns[6:]:
    #print(column)
    results[column] = pd.to_numeric(results[column], errors = 'coerce')


In [57]:
results

,participant_id,session_id,run_id,age,sex,site,gm_vol,wm_vol,csf_vol,gm_ivc_ratio,...,cov_gm_pvc0,cov_gm_pvc2,cov_wm_pvc0,cov_wm_pvc2,cov_aca_pvc0,cov_mca_pvc0,cov_pca_pvc0,cov_aca_pvc2,cov_mca_pvc2,cov_pca_pvc2
0,sub-0001_1_ses-1_run-1,ASL_1,1,43.49,M,TOP,0.71736,0.52803,0.31812,0.45881,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-0002_1_ses-1_run-1,ASL_1,1,38.30,F,TOP,0.72383,0.62394,0.25673,0.45112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub-0019_1_ses-1_run-1,ASL_1,1,32.30,M,TOP,0.71224,0.53295,0.33594,0.45046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub-0020_1_ses-1_run-1,ASL_1,1,21.97,F,TOP,0.67042,0.43458,0.25974,0.49124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub-0022_1_ses-1_run-1,ASL_1,1,37.52,F,TOP,0.63666,0.46938,0.22144,0.47960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,sub-1163_1_ses-1_run-1,ASL_1,1,19.06,F,TOP,0.78201,0.49489,0.24608,0.51347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527,sub-1165_1_ses-1_run-1,ASL_1,1,33.86,F,TOP,0.71537,0.48601,0.27123,0.48578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528,sub-1167_1_ses-1_run-1,ASL_1,1,33.92,M,TOP,0.73473,0.53549,0.35985,0.45074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529,sub-1168_1_ses-1_run-1,ASL_1,1,45.31,M,TOP,0.71094,0.60974,0.37480,0.41931,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:

def highlight_cols(s):
    color = 'red'
    return 'background-color: %s' % color




def find_outliers_by_list(dataframe, column_list):
    """
    This function finds the outliers in terms of anything outside two
    standard deviations
    from the mean on a list of specific specific column,
    then returns these rows of the dataframe.
    :param dataframe: whole dataframe on dataset
    :type dataframe: ~pandas.DataFrame
    :param column_list: list of relevant columns
    :type column_list: list
    :returns: dataframe of outliers
    :rtype: ~pandas.DataFrame
    """
    outlier_frames = []
    for column_n in column_list:
        mean = dataframe[column_n].mean()
        std = dataframe[column_n].std()
        values = dataframe[column_n].abs() - abs(mean + 2 * std)
        outliers = dataframe[values > 0]
        #outliers.style.applymap(highlight_cols, subset=pd.IndexSlice[:, [column_n]])
        #outliers.style.set_properties(subset=[column_n], **{'background-color': 'green'})
        outlier_frames.append(outliers)
        
    outlier_super = pd.concat(outlier_frames)
    return outlier_super


def check_sex_dimorph_expectations(dataframe):
    """
    This function checks that men
    as expected have larger brains than women
    in a given dataframe.
    """
    ladies = dataframe[dataframe['sex'] == 'F']
    men = dataframe[dataframe['sex'] == 'M']
    print('You have', len(ladies)/len(men), 'times as many ladies than men')
    if ladies.gm_vol.mean() < men.gm_vol.mean():
        print('As expected men have larger grey matter')
    if ladies.wm_vol.mean() < men.wm_vol.mean():
        print('As expected men have larger white matter')
    if ladies.gm_vol.mean() >= men.gm_vol.mean():
        print(
            'Caution, average female grey matter may be \
                  at similar or larger size than men'
        )
    if ladies.wm_vol.mean() >= men.wm_vol.mean():
        print(
            'Caution, average female white matter may be \
                  at similar or larger size than men'
        )
    if ladies.gm_vol.mean() >= men.gm_vol.mean() \
            or ladies.wm_vol.mean() >= men.wm_vol.mean():
        bad_data = dataframe
    else:
        bad_data = 0
    return bad_data

In [59]:
check_sex_dimorph_expectations(results)

You have 0.8631578947368421 times as many ladies than men
As expected men have larger grey matter
As expected men have larger white matter


0

In [60]:
len(results.columns.to_list())

76

In [61]:
# maybe examine graphs?
sep.relate_columns_graphs(results, 'age')

ValueError: Unable to parse string "sub-0001_1_ses-1_run-1" at position 0

ValueError: Image size of 1000x76000 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1000x76000 with 1 Axes>

In [ ]:
find_outliers_by_list(results, results.columns.to_list()[6:])

## Save off file

In [ ]:
filepath = '../open_work/internal_results' 
filename = os.path.join(filepath,'top_stitched_conformed.csv') 
if not os.path.exists(filepath):
    # if filder doesn't exist, create it
    os.makedirs(filepath)
results.to_csv(filename)  